## Preparations

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict


In [3]:
# change directory to src
# import os
os.chdir('../src')
from function_mapping import function_mapping

In [4]:
df_path = "../prompts_and_responses_02/responses_02.csv"
df = pd.read_csv(df_path, sep=';')

In [5]:
len(df)

70

In [6]:
# exclude rows with "summary_flag" == True
df = df[df["summary_flag"] == False]
len(df)

70

In [7]:
# to avoid exceptions for None values, remove the columns 'parameter_measurements' and 'used_measurements' from df
df = df.drop(columns=['parameter_measurements', 'used_measurements'])
df.head()

,ID,CONTROL_PROMPT_ID,user_prompt,LLM,intended_steps,steps,summary_flag,completion,correct_functions,correct_params,...,parameter_dish_name,used_dish_name,function_image_classification,used_image_classification,parameter_image_url,used_image_url,function_compute_shopping_list,used_compute_shopping_list,parameter_items_list,used_items_list
0,0,0,I would like to make a salad. I prefer Greek c...,gpt-3.5-turbo,1,1,False,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,I would like to make a salad. I prefer Greek c...,gpt-4-turbo,1,1,False,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5,Which wine pairs well with a cheese burger?,gpt-3.5-turbo,1,1,False,True,True,True,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5,Which wine pairs well with a cheese burger?,gpt-4-turbo,1,1,False,True,True,True,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,6,I have a bottle of Shiraz at home. Please sugg...,gpt-3.5-turbo,1,2,False,True,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Excel with manual annotations to CSV

In [9]:
# # load xlsx file
# df_xlsx = pd.read_excel("/Users/mayte/GitHub/openAI_experiments/prompts_and_responses_02/responses_02.xlsx")
# df_xlsx = df_xlsx[df_xlsx["summary_flag"] == False]
# df_xlsx.head(2)

In [10]:
# df = df_xlsx.copy(deep=True)

In [11]:
# df.head(2)

In [12]:
# # delete df.database column
# df = df.drop(columns=["database"])


In [13]:
# for col in df.columns:
#     print(col, df[col].dtype)


In [14]:
# # change steps and intended steps to int:
# df["steps"] = df["steps"].astype(int)
# df["intended_steps"] = df["intended_steps"].astype(int)
# # change the following to boolean values:
#     # summary_flag float64
#     # completion float64
#     # correct_functions float64
#     # correct_params float64
#     # correct_interpretation float64
#     # external_error float64
#     # api_output_correct float64
#     # caused_error float64
#     # corrected_error float64
# df["summary_flag"] = df["summary_flag"].astype(bool)
# df["completion"] = df["completion"].astype(bool)
# df["correct_functions"] = df["correct_functions"].astype(bool)
# df["correct_params"] = df["correct_params"].astype(bool)
# df["correct_interpretation"] = df["correct_interpretation"].astype(bool)
# df["external_error"] = df["external_error"].astype(bool)
# df["api_output_correct"] = df["api_output_correct"].astype(bool)
# df["caused_error"] = df["caused_error"].astype(bool)
# df["corrected_error"] = df["corrected_error"].astype(bool)


In [15]:
# # change domain and error to object
# # df["domain"] = df["domain"].astype(object)
# df["error"] = df["error"].astype(object)


In [16]:
# # save as csv
# df.to_csv("../prompts_and_responses_02/responses_02.csv",index=False, sep=";") # saving the dataframe as a csv file

## Error Analysis of Error logs (in message variable)

In [17]:
# import ast
# i = 0

In [18]:
# df_errors = df[df["caused_error"] == True]
# len(df_errors)

In [19]:
# i += 1
# print(f"{i}/{len(df_errors)}")
# print(df_errors.iloc[i]["LLM"])
# print(df_errors.iloc[i]["ID"])
# print("")
# messages_string = df_errors.iloc[i]["messages"]
# try:
#     messages = ast.literal_eval(messages_string)
#     print(f"Successfully parsed messages.")
# except ValueError as e:
#     print(f"Error parsing messages: {e}")
#     messages = []  # default to an empty list if parsing fails

# # Now you can iterate over messages if it's properly parsed
# for message in messages:
#     if message['role'] == 'system':
#         print(f"Role: {message['role']}")
#         if 'name' in message:
#             print(f"Function Name: {message['name']}")
#         print(f"Content: {message['content']}\n")

* 14 prompts that caused errors, some of them where the LLM made the same (kind of) mistake several times; (number of errors restricted to 3 before exiting the loop for chosing functions and parameters)  
* Date halluzination:
    * 8 fruit_and_veg
    * 4 fridge
    * wine_selection_or_offers
* Missing required parameter in find_recipe: 9  + 1 more issue with find_recipe: overall 10

## (Original) Analysis Notebook

In [20]:
# 41 of all rows have completion correct_functions	correct_params	correct_interpretation all set to True
columns_all_true = ['completion', 'correct_functions', 'correct_params', 'correct_interpretation']

# Create a Boolean mask where any of the specified columns have the value "false"
mask_all_true= (df[columns_all_true] == True).all(axis=1)
indexes_all_true = df.index[mask_all_true].tolist()
ratio_all_true = len(indexes_all_true) / len(df)
print(f"Rows that completed the task (no issues): {ratio_all_true:.2%}")

Rows that completed the task (no issues): 74.29%


In [21]:
# ratio of rows that have completion set to True to rows that have completion set to False
mask_completion_false = (df['completion'] == False)
mask_completion_true = (df['completion'] == True)
indexes_completion_false = df.index[mask_completion_false].tolist()
indexes_completion_true = df.index[mask_completion_true].tolist()
ratio_completion_false = len(indexes_completion_true) / len(indexes_completion_false)
print(f"Prompts that were completed vs not completed: {len(indexes_completion_true)}:{len(indexes_completion_false)}")

Prompts that were completed vs not completed: 55:15


In [22]:
# delete all rows with "gtp-4-turbo" in column "LLM"
df_sorted = df.copy()
print(len(df_sorted))
# use unique combinations of the two columns 'user_prompt' and 'LLM'; if there are multiple rows with the same combination, check if one of them has 'completion' set to True
# Sort the DataFrame such that the True values in 'completion' come last for each group
df_sorted = df_sorted .sort_values(by=['user_prompt', 'LLM', 'completion'], ascending=[True, True, False])

# Use drop_duplicates to keep the last occurrence, which will be the one where 'completion' is True if it exists
df_unique_sorted = df_sorted.drop_duplicates(subset=['user_prompt', 'LLM'], keep='last')
len(df_unique_sorted)

70


70

In [23]:
# keep the columns: "user_prompt", "steps",  "LLM", "completion"
df_prompts_results = df_unique_sorted[["user_prompt", "steps",  "LLM", "completion"]]
df_prompts_results.head(2)

,user_prompt,steps,LLM,completion
6,Based on the image at https://2.bp.blogspot.co...,2,gpt-3.5-turbo,True
7,Based on the image at https://2.bp.blogspot.co...,2,gpt-4-turbo,True


In [24]:
# count how many times prompts appear once or twice
df_prompts_results['user_prompt'].value_counts().value_counts()

count
2    35
Name: count, dtype: int64

In [25]:
# show the prompts that appears once
df_prompts_results['user_prompt'].value_counts()[df_prompts_results['user_prompt'].value_counts() == 1]

Series([], Name: count, dtype: int64)

### Analysis Label "complete"

In [26]:
# # # or start reshere and load
# df_merged = pd.read_csv('unique_prompts_evaluation.csv', sep=';') # load the csv file with filtered data and intended steps
# df_merged = pd.read_csv('df_merged_analysis.csv', sep=';')
df_prompts_results = df.copy()

In [27]:
# Group by 'LLM' and 'completion', then count each group (use plotly)
df_counts = df_prompts_results.groupby('LLM')['completion'].value_counts().reset_index(name='count')
fig_completion = px.bar(df_counts, x='LLM', y='count', color='completion', title='Completion Counts per LLM',
            labels={'count': 'Number of Completions', 'completion': 'Completion Status'},
            barmode='group',
            color_discrete_map={True: 'teal', False: 'lightcoral'},
            # pattern_shape='completion',
            )
fig_completion.show()

In [28]:
df_prompts_results.head(2)

,ID,CONTROL_PROMPT_ID,user_prompt,LLM,intended_steps,steps,summary_flag,completion,correct_functions,correct_params,...,parameter_dish_name,used_dish_name,function_image_classification,used_image_classification,parameter_image_url,used_image_url,function_compute_shopping_list,used_compute_shopping_list,parameter_items_list,used_items_list
0,0,0,I would like to make a salad. I prefer Greek c...,gpt-3.5-turbo,1,1,False,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,I would like to make a salad. I prefer Greek c...,gpt-4-turbo,1,1,False,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Group by 'user_prompt' and list completion values
grouped = df_prompts_results.groupby('user_prompt').agg({
    'completion': list,
    'intended_steps': 'first'  # All intended_steps values should be the same per prompt
})
grouped.head(3)

,completion,intended_steps
user_prompt,,
"Based on the image at https://2.bp.blogspot.com/-TtabQoqmHn8/VExgR9YejVI/AAAAAAAAAMk/Ikic-vGdB30/s1600/20141025_221431.jpg, please identify the dish and find a recipe.","[True, True]",2
"Based on the image at https://2.bp.blogspot.com/-TtabQoqmHn8/VExgR9YejVI/AAAAAAAAAMk/Ikic-vGdB30/s1600/20141025_221431.jpg, please identify the dish and suggest a recipe. Please add all ingredients to my shopping list.","[True, True]",3
"Based on the image at https://cjoloughlin.ie/wp-content/uploads/2015/09/shutterstock_142426168.jpg, please identify the dish and find a recipe.","[True, True]",2


In [30]:

# Initialize dictionaries to hold results and intended_steps
category_data = {
    'both_true': [],
    'both_false': [],
    'only_3_5': [],
    'only_4': []
}

# Classify prompts
for prompt, details in grouped.iterrows():
    completions = details['completion']
    steps = details['intended_steps']

    if completions == [True, True]:
        category_data['both_true'].append(steps)
    elif completions == [False, False]:
        category_data['both_false'].append(steps)
    elif completions == [True, False]:
        category_data['only_3_5'].append(steps)
    elif completions == [False, True]:
        category_data['only_4'].append(steps)

# Print number of prompts and their intended steps
for category, steps in category_data.items():
    print(f"Prompts in {category}: {len(steps)}")
    print(f"Intended steps: {steps}")
    print()


Prompts in both_true: 23
Intended steps: [2, 3, 2, 3, 1, 4, 2, 3, 2, 1, 2, 2, 3, 2, 1, 3, 3, 2, 2, 3, 2, 1, 4]

Prompts in both_false: 3
Intended steps: [4, 3, 4]

Prompts in only_3_5: 2
Intended steps: [4, 4]

Prompts in only_4: 7
Intended steps: [3, 4, 4, 4, 4, 3, 5]



In [31]:
# visualize the difficulty (= intended steps) vs the completion status
data = {
    'both_true': category_data['both_true'],
    'both_false': category_data['both_false'],
    'only_3_5': category_data['only_3_5'],
    'only_4': category_data['only_4']
}
# Normalize lengths by converting lists to pandas Series and allowing NaN values
max_length = max(len(v) for v in data.values())
for key, value in data.items():
    data[key] = pd.Series(value).reindex(range(max_length))

# Create DataFrame
df_steps = pd.DataFrame(data)


# Display DataFrame
df_steps.head(6)

,both_true,both_false,only_3_5,only_4
0,2,4.0,4.0,3.0
1,3,3.0,4.0,4.0
2,2,4.0,NaN,4.0
3,3,NaN,NaN,4.0
4,1,NaN,NaN,4.0
5,4,NaN,NaN,3.0


In [32]:
# define colour scheme:
color_discrete_map={
    'both_true': 'mediumaquamarine',
    'both_false': 'indianred', 
    'only_3_5': 'darkslateblue',
    'only_4': 'mediumpurple',
    'True': 'seagreen',
    'False': 'palevioletred',
}

In [33]:
# Reshape the DataFrame from wide to long format
df_long = df_steps.melt(var_name='Category', value_name='Intended Steps')
df_long.head(2)
# Create a box plot with plotly
fig_box = px.box(df_long, x='Category', y='Intended Steps',
                 title='Distribution of Intended Steps by Completion Status',
                 labels={'Intended Steps': 'Number of Intended Steps'},
                 color='Category',
                 color_discrete_map=color_discrete_map)
# Update the axis titles
fig_box.update_layout(
    xaxis_title='Category: Prompt Completion Status',
    yaxis_title='Intended Steps'
)
# add labels to categories:
fig_box.update_xaxes(tickvals=[0, 1, 2, 3], ticktext=['both completed', 'both not completed', 'only completed by 3.5', 'only completed by 4'])
# y-axis only integers
fig_box.update_yaxes(tickmode='linear')
# remove legend
fig_box.update_layout(showlegend=False)
fig_box.show()

In [34]:
# Calculate frequencies of each intended step within each category
df_count = pd.DataFrame({col: df_steps[col].value_counts() for col in df_steps.columns})
df_count = df_count.fillna(0)  # Replace NaN with 0 for frequencies
df_count.sort_index(inplace=True)  # Sort by intended steps for consistent plotting

# Print the frequency DataFrame
df_count.head(2)

,both_true,both_false,only_3_5,only_4
1.0,4.0,0.0,0.0,0.0
2.0,10.0,0.0,0.0,0.0


In [35]:
# create 4 subgraphs (fig, axes) with plotly
# each subgraph should contain information about one category only
# use plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig_sub = make_subplots(rows=1, cols=4, subplot_titles=("Completed by both", "Completed by none", "Only 3.5-Turbo", "Only 4-Turbo"))
# plot the number of intended steps on the x-axis and the frequency on the y-axis
fig_sub.add_trace(go.Bar(x=df_count.index, y=df_count['both_true'], name='Completed by both', marker_color=color_discrete_map['both_true']), row=1, col=1)
fig_sub.add_trace(go.Bar(x=df_count.index, y=df_count['both_false'], name='Completed by none', marker_color=color_discrete_map['both_false']), row=1, col=2)
fig_sub.add_trace(go.Bar(x=df_count.index, y=df_count['only_3_5'], name='only completed by GPT 3.5-Turbo', marker_color=color_discrete_map['only_3_5']), row=1, col=3)
fig_sub.add_trace(go.Bar(x=df_count.index, y=df_count['only_4'], name='only completed by GPT 4-TUrbo', marker_color=color_discrete_map['only_4']), row=1, col=4)
# use same y-axis scale for all subplots
fig_sub.update_yaxes(range=[0, 8])
# remove legend
fig_sub.update_layout(showlegend=False)
# Add title
fig_sub.update_layout(title_text="Distribution of Intended Steps by Completion Status")
# x-axis title

# keep x-axis title only once
fig_sub.update_xaxes(title_text="Intended Steps")
# y-axis title
fig_sub.update_yaxes(title_text="Frequency", row=1, col=1)


In [36]:
# produce the stacked graph with plotly
fig_stacked_plotly = px.bar(df_count, x=df_count.index, y=df_count.columns, title='Completion Status by Intended Steps',
             labels={'value': 'Frequency', 'index': 'Intended Steps', 'variable': 'Category'},
             barmode='stack',
             # change barmode to group
            # barmode='group',
            color_discrete_map=color_discrete_map,
            # add pattern shape to each colour
            # pattern_shape='Category',
            # text_auto=True,
            )
# x-axis labels only integers
fig_stacked_plotly.update_xaxes(type='category')
# change names in legend:
# change manually:
fig_stacked_plotly.update_traces(name='Completed by both', selector=dict(name='both_true'))
fig_stacked_plotly.update_traces(name='Completed by none', selector=dict(name='both_false'))
fig_stacked_plotly.update_traces(name='Only by 3.5', selector=dict(name='only_3_5'))
fig_stacked_plotly.update_traces(name='Only by 4', selector=dict(name='only_4'))

fig_stacked_plotly.show()

In [37]:
## Create bar plot per LLM

# filter for GPT-3.5-Turbo Data
df_gpt_3_5 = df_prompts_results[df_prompts_results['LLM'] == 'gpt-3.5-turbo']
df_gpt_3_5_counts = df_gpt_3_5.groupby('intended_steps')['completion'].value_counts().unstack().fillna(0) # Count the number of completions per intended steps
df_gpt_3_5_counts.reset_index(inplace=True) # Reset the index
df_gpt_3_5_counts.columns = ['intended_steps', 'False', 'True']
df_gpt_3_5_counts = df_gpt_3_5_counts.astype(int) # set values to integers

In [38]:
df_gpt_4 = df_prompts_results[df_prompts_results['LLM'] == 'gpt-4-turbo']
df_gpt_4_counts = df_gpt_4.groupby('intended_steps')['completion'].value_counts().unstack().fillna(0) # Count the number of completions for each intended step
df_gpt_4_counts.reset_index(inplace=True) # Reset the index
df_gpt_4_counts = df_gpt_4_counts.astype(int)  # Convert the counts to integers
# rename columns True to "True" and False to "False
df_gpt_4_counts.columns = ['intended_steps', 'False', 'True']
df_gpt_4_counts.head(10)

,intended_steps,False,True
0,1,0,4
1,2,0,10
2,3,1,9
3,4,3,7
4,5,0,1


In [39]:
fig_intended_steps_gpt_3_5 = px.bar(df_gpt_3_5_counts, x=df_gpt_3_5_counts.intended_steps, y=df_gpt_3_5_counts.columns, title='Completion Status by Intended Steps',
             labels={'value': 'Frequency', 'index': 'Intended Steps', 'variable': 'Category'},
            barmode='group', # change barmode to group
            color_discrete_map=color_discrete_map,
            text_auto=True,
            )
fig_intended_steps_gpt_3_5.update_xaxes(title_text="Intended Steps")
fig_intended_steps_gpt_3_5.show()

In [40]:
fig_intended_steps_gpt_4 = px.bar(df_gpt_4_counts, x=df_gpt_4_counts.intended_steps, y=df_gpt_4_counts.columns, title='Completion Status by Intended Steps',
             labels={'value': 'Frequency', 'index': 'Intended Steps', 'variable': 'Category'},
            barmode='group', # change barmode to group
            color_discrete_map=color_discrete_map,
            text_auto=True,
            )
fig_intended_steps_gpt_4.update_xaxes(title_text="Intended Steps")
fig_intended_steps_gpt_4.show()

In [41]:
# create two subplots with plotly
fig_intended_steps = make_subplots(
    rows=1, cols=2,
    subplot_titles=('GPT-3.5-Turbo', 'GPT-4-Turbo'),
    shared_yaxes=True
)

# Add the first bar chart to the first column of the subplot
# Plot for GPT-3.5
fig_intended_steps.add_trace(
    go.Bar(
        x=df_gpt_3_5_counts['intended_steps'],
        y=df_gpt_3_5_counts['True'],
        name='Complete',
        text=df_gpt_3_5_counts['True'],
        textposition='auto',
        marker_color=color_discrete_map['True']
    ),
    row=1, col=1
)
fig_intended_steps.add_trace(
    go.Bar(
        x=df_gpt_3_5_counts['intended_steps'],
        y=df_gpt_3_5_counts['False'],
        name = 'Incomplete',
        text=df_gpt_3_5_counts['False'],
        textposition='auto',
        marker_color=color_discrete_map['False'],
    ),
    row=1, col=1
)

# Plot for GPT-4
fig_intended_steps.add_trace(
    go.Bar(
        x=df_gpt_4_counts['intended_steps'],
        y=df_gpt_4_counts['True'],
        name='Complete',
        text=df_gpt_4_counts['True'],
        textposition='auto',
        showlegend=False,
        marker_color=color_discrete_map['True']
    ),
    row=1, col=2
)
fig_intended_steps.add_trace(
    go.Bar(
        x=df_gpt_4_counts['intended_steps'],
        y=df_gpt_4_counts['False'],
        name = 'Incomplete',
        text=df_gpt_4_counts['False'],
        textposition='auto',
        showlegend=False,
        marker_color=color_discrete_map['False'],
    ),
    row=1, col=2
)
# Update xaxes and yaxes properties
fig_intended_steps.update_xaxes(title_text="Intended Steps", row=1, col=1)
fig_intended_steps.update_xaxes(title_text="Intended Steps", row=1, col=2)
fig_intended_steps.update_yaxes(title_text="Frequency", row=1, col=1)

# set title
fig_intended_steps.update_layout(title_text="Completion Status by Intended Steps")
fig_intended_steps.show()

In [42]:
df_gpt_3_5_counts.columns

Index(['intended_steps', 'False', 'True'], dtype='object')

In [43]:
# create "confusion matrix"
# Data dictionary
data = {
    'completed': [len(category_data['both_true']), len(category_data['only_3_5'])], # by GPT 3.5-turbo
    'not completed': [len(category_data['only_4']), len(category_data['both_false'])] # by GPT 3.5-turbo
}

# Creating the DataFrame
confusion_matrix = pd.DataFrame(data, index=['completed', 'not completed']) # by GPT 4

# Print the confusion matrix
print(confusion_matrix)

               completed  not completed
completed             23              7
not completed          2              3


In [44]:
# create heatmap with plotly of the confusion matrix
fig_heatmap = px.imshow(confusion_matrix, title='Confusion Matrix',
             labels=dict(x='GPT 3.5-turbo', y='GPT 4-Turbo', color='Number of Prompts'),
             text_auto=True,
             color_continuous_scale='blugrn')
# change colourscheme to blues

fig_heatmap.show()

# Analysis of the Responses on the level of individual functions and parameters

#### Preparations

In [46]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import defaultdict
# load df_merged_analysis.csv
# change directory to current directory
import os
# os.chdir('../prompts_and_responses')
# df = pd.read_csv('df_merged_analysis_02.csv', sep=';') #TODO: maybe change file?
df.head(2)


,ID,CONTROL_PROMPT_ID,user_prompt,LLM,intended_steps,steps,summary_flag,completion,correct_functions,correct_params,...,parameter_dish_name,used_dish_name,function_image_classification,used_image_classification,parameter_image_url,used_image_url,function_compute_shopping_list,used_compute_shopping_list,parameter_items_list,used_items_list
0,0,0,I would like to make a salad. I prefer Greek c...,gpt-3.5-turbo,1,1,False,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,I would like to make a salad. I prefer Greek c...,gpt-4-turbo,1,1,False,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
function_map = function_mapping(df)

In [48]:
# Print the structure to verify
for function, details in function_map.items():
    print(f"Function: {function}, Details: {details}")

Function: function_fruit_and_veg_offers, Details: {'parameters': ['parameter_date_fruit_and_veg_offers'], 'used_parameters': ['used_date_fruit_and_veg_offers'], 'used': 'used_fruit_and_veg_offers'}
Function: function_wine_selection_or_offers, Details: {'parameters': ['parameter_date_wine_selection_or_offers', 'parameter_wine_type'], 'used_parameters': ['used_date_wine_selection_or_offers', 'used_wine_type'], 'used': 'used_wine_selection_or_offers'}
Function: function_fridge_items, Details: {'parameters': ['parameter_date_fridge_items'], 'used_parameters': ['used_date_fridge_items'], 'used': 'used_fridge_items'}
Function: function_date, Details: {'parameters': [], 'used_parameters': [], 'used': 'used_date'}
Function: function_get_recipe_info, Details: {'parameters': ['parameter_recipe_id', 'parameter_instructions', 'parameter_time', 'parameter_ingredients'], 'used_parameters': ['used_recipe_id', 'used_instructions', 'used_time', 'used_ingredients'], 'used': 'used_get_recipe_info'}
Funct

#### No Filtering for LLM or intended steps

In [49]:
def function_statistics(df, function_map, function_name_list):
    """
    TODO
    """
    # create df to store the results
    df_results = pd.DataFrame(columns=['function', 'intended_uses', 'potential_uses', 'use_not_possible', 'causing_errors', 'unneccesary_uses'])
    for function_name in function_name_list:
        column_function = function_name
        column_used_function = function_map[function_name]['used']

        # Calculate totals for the function's usage
        intended_uses = df[column_function].isin([1, -1, 0]).sum()
        potential_uses = (df[column_function] == 1).sum()
        use_not_possible = (df[column_function] == 0).sum()
        causing_errors = (df[column_function] == -1).sum()
        unneccesary_uses = (df[column_used_function] == -2).sum()

        # Append the results to the DataFrame
        next_index = len(df_results)
        df_results.loc[next_index] = {
            'function': function_name[9:],
            'intended_uses': intended_uses,
            'potential_uses': potential_uses,
            'use_not_possible': use_not_possible,
            'causing_errors': causing_errors,
            'unneccesary_uses': unneccesary_uses}
    # create last row with totals
    df_results.loc[next_index + 1] = {
        'function': 'Total',
        'intended_uses': df_results['intended_uses'].sum(),
        'potential_uses': df_results['potential_uses'].sum(),
        'use_not_possible': df_results['use_not_possible'].sum(),
        'causing_errors': df_results['causing_errors'].sum(),
        'unneccesary_uses': df_results['unneccesary_uses'].sum()
    }
    return df_results




In [50]:
# all function_names
function_name_list = list(function_map.keys())
# Calculate statistics for all functions
df_results = function_statistics(df, function_map, function_name_list)
df_results.tail(20)

,function,intended_uses,potential_uses,use_not_possible,causing_errors,unneccesary_uses
0,fruit_and_veg_offers,20,20,0,0,0
1,wine_selection_or_offers,10,10,0,0,0
2,fridge_items,8,8,0,0,2
3,date,38,38,0,0,0
4,get_recipe_info,32,31,1,0,0
5,find_recipe,52,52,0,0,0
6,dish_pairing_for_wine,10,10,0,0,0
7,wine_pairing,10,8,2,0,2
8,image_classification,14,14,0,0,0
9,compute_shopping_list,8,8,0,0,0


In [53]:
# # df_results remove column 'causing_errors' for logging
# df_results = df_results.drop(columns=['causing_errors'])
# df_results.tail(20)

,function,intended_uses,potential_uses,use_not_possible,unneccesary_uses
0,fruit_and_veg_offers,20,20,0,0
1,wine_selection_or_offers,10,10,0,0
2,fridge_items,8,8,0,2
3,date,38,38,0,0
4,get_recipe_info,32,31,1,0
5,find_recipe,52,52,0,0
6,dish_pairing_for_wine,10,10,0,0
7,wine_pairing,10,8,2,2
8,image_classification,14,14,0,0
9,compute_shopping_list,8,8,0,0


In [51]:
def calculate_function_accuracy(df, function_map, function_name):
    
    function_col = function_name
    used_function_col = function_map[function_name]['used']
    parameters = function_map[function_name]['parameters']
    used_parameters = function_map[function_name]['used_parameters']

    # Rows where function was intended to be used and was used (needed to check parameter use)
    valid_function_df = df[((df[function_col] == 1) | (df[function_col] == -1)) & (df[used_function_col] == 1)]
    # exclude rows for testing the parameter where the function was never called (! but counted for all_func)

    # Calculate total correct uses
    total_correct_uses = valid_function_df.apply(
        lambda row: all(row[used_param] == 1 if row[param] in [1, -1] else True for param, used_param in zip(parameters, used_parameters)),
        axis=1
    ).sum()
    # count all function calls (function 
    all_func = ((df[function_col] == 1) | (df[function_col] == -1)).sum()

    accuracy_function = total_correct_uses / all_func if all_func > 0 else None
    return accuracy_function


In [52]:
def calculate_parameter_accuracy(df, function_col, param_col, used_param_col):
    # Filter only rows where the function was used or intended to be used
    all_param = (((df[function_col] == 1) | (df[function_col] == -1)) & ((df[param_col] == 1) | (df[param_col] == -1))).sum()
    # Filter only rows where the function was used or intended to be used and the parameter was used correctly
    correct_param = (((df[function_col] == 1) | (df[function_col] == -1)) & ((df[param_col] == 1) | (df[param_col] == -1)) & (df[used_param_col] == 1)).sum()
    # Calculate accuracy as the ratio of correct parameter usage to all intended parameter usage
    accuracy_param = correct_param / all_param if all_param > 0 else None # Avoid division by zero
    return accuracy_param


In [53]:
# Example usage for a function and its parameters
function_name = 'function_find_recipe'
param_col = 'parameter_time'
used_param_col = 'used_time'
accuracy_param = calculate_parameter_accuracy(df, function_name, param_col, used_param_col)
print(f"Parameter accuracy for parameter '{param_col[10:]}': {accuracy_param:.2%}")


accuracy_function = calculate_function_accuracy(df, function_map, function_name)
print(f"Function accuracy for '{function_name}': {accuracy_function:.2%}")

Parameter accuracy for parameter 'time': 100.00%
Function accuracy for 'function_find_recipe': 76.92%


In [54]:
def plot_all_parameter_accuracies(df, function_map):
    """
    Plots the accuracy of each parameter for all functions using the function_map dictionary.

    Args:
    df (DataFrame): The DataFrame containing the data.
    function_map (dict): Dictionary mapping functions to their parameters and used parameters.
    """

    fig_params = go.Figure()

    # Iterate over all functions in the function_map
    for function_name, details in function_map.items():
        function_col = function_name  # The column name for the function
        used_function_col = details['used']  # The column name for used function
        parameters = details['parameters']
        used_parameters = details['used_parameters']

        # Calculate accuracies for each parameter of the current function
        for param, used_param in zip(parameters, used_parameters):
            accuracy_param = calculate_parameter_accuracy(df, function_col, param, used_param)
            trimmed_param = param.replace('parameter_', '')  # Trimming 'parameter_' for better display
            function_display_name = function_name.replace('function_', '')  # Trimming 'function_' for display
            fig_params.add_trace(go.Bar(
                x=[f"{function_display_name} - {trimmed_param}"],
                y=[accuracy_param],
                name=f"{function_display_name} - {trimmed_param}",
                text=f"{accuracy_param:.2}",
                # text position upwards for better visibility
                textposition='inside',
                showlegend=True
            ))

    # Update layout of the plot
    fig_params.update_layout(
        title="Accuracy of Parameters",
        xaxis_title="Function - Parameter",
        yaxis_title="Accuracy",
        # yaxis=dict(tickformat="."),
        barmode='group'
    )
    return fig_params


In [55]:
# Example Usage accuracy of all parameters (for all functions)
fig_accuracy_all_parameters = plot_all_parameter_accuracies(df, function_map)
fig_accuracy_all_parameters.show()

In [56]:
# plot accuracy for all functions
def plot_all_function_accuracies(df, function_map):
    """
    Plots the accuracy of each function using the function_map dictionary.

    Args:
    df (DataFrame): The DataFrame containing the data.
    function_map (dict): Dictionary mapping functions to their parameters and used parameters.
    """

    fig_functions = go.Figure()

    # Iterate over all functions in the function_map
    for function_name in function_map.keys():
        accuracy_function = calculate_function_accuracy(df, function_map, function_name)
        function_display_name = function_name.replace('function_', '')  # Trimming 'function_' for display
        fig_functions.add_trace(go.Bar(
            x=[function_display_name],
            y=[accuracy_function],
            name=function_display_name,
            text=f"{accuracy_function:.2}",
            showlegend=True
        ))

    # Update layout of the plot
    fig_functions.update_layout(
        title="Accuracy of Functions",
        xaxis_title="Function",
        yaxis_title="Accuracy",
        # yaxis=dict(tickformat=".0")
        barmode='group'
    )
    return fig_functions


In [57]:
# Example Usage accuarcy of all functions
fig_accuracy_functions = plot_all_function_accuracies(df, function_map)
fig_accuracy_functions.show()

#### On Parameter Level (per LLM / per intended steps)

In [58]:
def calculate_parameter_accuracy_per_llm(df, function_map):
    accuracies = {}

    # Group the dataframe by 'LLM' for calculation
    grouped = df.groupby('LLM')

    # Calculate accuracy for each parameter within each LLM group
    for llm, group in grouped:
        accuracy_per_parameter = {}
        for function_name, details in function_map.items():
            for param, used_param in zip(details['parameters'], details['used_parameters']):
                accuracy = calculate_parameter_accuracy(group, function_name, param, used_param)
                accuracy_per_parameter[param] = accuracy
        
        accuracies[llm] = accuracy_per_parameter

    # Calculate total accuracies across all LLMs
    accuracy_per_parameter = {}
    for function_name, details in function_map.items():
        for param, used_param in zip(details['parameters'], details['used_parameters']):
            accuracy = calculate_parameter_accuracy(df, function_name, param, used_param)
            accuracy_per_parameter[param] = accuracy
    accuracies['Total'] = accuracy_per_parameter

    # Convert the dictionary to a DataFrame for easier handling
    accuracies_df = pd.DataFrame(accuracies)

    return accuracies_df

# Assuming calculate_parameter_accuracy is defined correctly and function_map is available
# Calculate the parameter accuracies per LLM
df_parameter_accuracy_per_llm = calculate_parameter_accuracy_per_llm(df, function_map)
df_parameter_accuracy_per_llm


,gpt-3.5-turbo,gpt-4-turbo,Total
parameter_date_fruit_and_veg_offers,0.700000,1.000000,0.850000
parameter_date_wine_selection_or_offers,1.000000,1.000000,1.000000
parameter_wine_type,1.000000,1.000000,1.000000
parameter_date_fridge_items,0.750000,0.750000,0.750000
parameter_recipe_id,0.916667,1.000000,0.961538
parameter_instructions,0.750000,0.750000,0.750000
parameter_time,1.000000,1.000000,1.000000
parameter_ingredients,1.000000,1.000000,1.000000
parameter_query,0.800000,0.961538,0.882353
parameter_diet,1.000000,1.000000,1.000000


In [59]:
# create a copy of with rounded values
# create a copy of with rounded values
temp_df = df_parameter_accuracy_per_llm.copy().round(2)
# transpose temp_df
temp_df = temp_df.transpose()
# change all parameter names to remove 'parameter_'
temp_df.columns = [col[10:] for col in temp_df.columns]
fig_accuracy_param_llm = px.imshow(temp_df,
                labels=dict(x="Parameter", y="LLM", color="Accuracy"),
                x=temp_df.columns,
                y=temp_df.index,
                aspect="auto",
                color_continuous_scale='blugrn',
                text_auto=True)  

fig_accuracy_param_llm.update_layout(
    title="Parameter Accuracy per LLM")
fig_accuracy_param_llm.update_xaxes(showgrid=False)
fig_accuracy_param_llm.update_yaxes(showgrid=False)
fig_accuracy_param_llm.show()

In [60]:
def calculate_parameter_accuracy_per_intended_step(df, function_map):
    accuracies = {}

    # Group the dataframe by 'intended_steps' for calculation
    grouped = df.groupby('intended_steps')

    # Calculate accuracy for each parameter within each LLM group
    for intended_step, group in grouped:
        accuracy_per_parameter = {}
        for function_name, details in function_map.items():
            for param, used_param in zip(details['parameters'], details['used_parameters']):
                accuracy = calculate_parameter_accuracy(group, function_name, param, used_param)
                accuracy_per_parameter[param] = accuracy
        
        accuracies[intended_step] = accuracy_per_parameter

    # Calculate total accuracies across all intended_steps
    accuracy_per_parameter = {}
    for function_name, details in function_map.items():
        for param, used_param in zip(details['parameters'], details['used_parameters']):
            accuracy = calculate_parameter_accuracy(df, function_name, param, used_param)
            accuracy_per_parameter[param] = accuracy
    accuracies['Total'] = accuracy_per_parameter

    # Convert the dictionary to a DataFrame for easier handling
    accuracies_df = pd.DataFrame(accuracies)
    # change type of column names to strings
    accuracies_df.columns = accuracies_df.columns.astype(str)

    return accuracies_df

# Assuming calculate_parameter_accuracy is defined correctly and function_map is available
# Calculate the parameter accuracies per LLM
df_parameter_accuracy_per_intended_step = calculate_parameter_accuracy_per_intended_step(df, function_map)
df_parameter_accuracy_per_intended_step

,1,2,3,4,5,Total
parameter_date_fruit_and_veg_offers,NaN,1.0,1.0,0.812500,NaN,0.850000
parameter_date_wine_selection_or_offers,NaN,1.0,1.0,1.000000,1.0,1.000000
parameter_wine_type,NaN,NaN,1.0,NaN,NaN,1.000000
parameter_date_fridge_items,NaN,1.0,1.0,0.000000,NaN,0.750000
parameter_recipe_id,1.0,1.0,1.0,1.000000,0.5,0.961538
parameter_instructions,NaN,1.0,1.0,0.000000,NaN,0.750000
parameter_time,NaN,NaN,1.0,NaN,NaN,1.000000
parameter_ingredients,1.0,1.0,1.0,1.000000,1.0,1.000000
parameter_query,1.0,1.0,1.0,0.666667,1.0,0.882353
parameter_diet,1.0,1.0,1.0,1.000000,NaN,1.000000


In [61]:
# change index to strings
# temp_df.index = temp_df.index.astype(str)
df_parameter_accuracy_per_intended_step.columns

Index(['1', '2', '3', '4', '5', 'Total'], dtype='object')

In [62]:
# create a copy of with rounded values
temp_df = df_parameter_accuracy_per_intended_step.copy().round(2)
# transpose temp_df
temp_df = temp_df.transpose()
# change all parameter names to remove 'parameter_'
temp_df.columns = [col[10:] for col in temp_df.columns]
fig_accuracy_param_steps = px.imshow(temp_df,
                labels=dict(x="Intended Steps", y="Parameters", color="Accuracy"),
                x=temp_df.columns,
                y=temp_df.index,
                aspect="auto",
                color_continuous_scale='blugrn',
                # round the values to 2 decimal place
                text_auto=True,

                )
fig_accuracy_param_steps.update_xaxes(showgrid=False)
fig_accuracy_param_steps.update_yaxes(showgrid=False)
fig_accuracy_param_steps.show()

#### Function vs LLM / intended steps

In [63]:
grouped = df.groupby('LLM')
grouped.head()

,Unnamed: 0,ID,CONTROL_PROMPT_ID,user_prompt,LLM,intended_steps,steps,summary_flag,completion,correct_functions,...,parameter_dish_name,used_dish_name,function_image_classification,used_image_classification,parameter_image_url,used_image_url,function_compute_shopping_list,used_compute_shopping_list,parameter_items_list,used_items_list
0,0,0,0,I would like to make a salad. I prefer Greek c...,gpt-3.5-turbo,1,1,False,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,0,I would like to make a salad. I prefer Greek c...,gpt-4-turbo,1,1,False,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,5,Which wine pairs well with a cheese burger?,gpt-3.5-turbo,1,1,False,True,True,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,5,Which wine pairs well with a cheese burger?,gpt-4-turbo,1,1,False,True,True,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,6,I have a bottle of Shiraz at home. Please sugg...,gpt-3.5-turbo,1,2,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,5,6,I have a bottle of Shiraz at home. Please sugg...,gpt-4-turbo,1,1,False,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,6,8,Based on the image at https://2.bp.blogspot.co...,gpt-3.5-turbo,2,2,False,True,True,...,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
7,7,7,8,Based on the image at https://2.bp.blogspot.co...,gpt-4-turbo,2,2,False,True,True,...,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
8,8,8,9,Using the image athttps://www.tasteforcooking....,gpt-3.5-turbo,2,2,False,True,True,...,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
9,9,9,9,Using the image athttps://www.tasteforcooking....,gpt-4-turbo,2,2,False,True,True,...,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [64]:
def calculate_function_accuracy_per_llm(df, function_map):
    accuracies = {}

    # Group the dataframe by 'LLM' for calculation
    grouped = df.groupby('LLM')

    # Iterate over all LLM groups
    for llm, group in grouped:
        # For each function, calculate its accuracy within the current LLM group
        accuracy_per_function = {}
        for function_name in function_map.keys():
            accuracy = calculate_function_accuracy(group, function_map, function_name)
            accuracy_per_function[function_name] = accuracy

        # Store the accuracies for each function within this LLM
        accuracies[llm] = accuracy_per_function

    accuracy_per_function = {}
    for function_name in function_map.keys():
        accuracy = calculate_function_accuracy(df, function_map, function_name)
        accuracy_per_function[function_name] = accuracy
    accuracies['Total'] = accuracy_per_function

    # Convert the dictionary to a DataFrame for easier handling
    accuracies_df = pd.DataFrame(accuracies)


    return accuracies_df

# Calculate the function accuracies per LLM
df_calculate_function_accuracy_per_llm = calculate_function_accuracy_per_llm(df, function_map)
df_calculate_function_accuracy_per_llm

,gpt-3.5-turbo,gpt-4-turbo,Total
function_fruit_and_veg_offers,0.700000,1.000000,0.850000
function_wine_selection_or_offers,1.000000,1.000000,1.000000
function_fridge_items,0.750000,0.750000,0.750000
function_date,0.789474,0.947368,0.868421
function_get_recipe_info,0.625000,0.933333,0.774194
function_find_recipe,0.653846,0.884615,0.769231
function_dish_pairing_for_wine,0.600000,1.000000,0.800000
function_wine_pairing,0.750000,1.000000,0.875000
function_image_classification,1.000000,1.000000,1.000000
function_compute_shopping_list,0.750000,1.000000,0.875000


In [65]:
# create a copy of with rounded values
temp_df = df_calculate_function_accuracy_per_llm.copy().round(2)
# transpose temp_df
temp_df = temp_df.transpose()
# change all parameter names to remove 'parameter_'
temp_df.columns = [col[9:] for col in temp_df.columns]
fig_accuracy_func_llm = px.imshow(temp_df,
                labels=dict(x="Function", y="LLM", color="Accuracy"),
                x=temp_df.columns,
                y=temp_df.index,
                aspect="auto",
                color_continuous_scale='blugrn',
                text_auto=True)  

fig_accuracy_func_llm.update_layout(
    title="Parameter Accuracy per LLM")
fig_accuracy_func_llm.update_xaxes(showgrid=False)
fig_accuracy_func_llm.update_yaxes(showgrid=False)
fig_accuracy_func_llm.show()

In [66]:
def calculate_function_accuracy_per_intended_step(df, function_map):
    accuracies = {}

    # Group the dataframe by 'intended_steps' for calculation
    grouped = df.groupby('intended_steps')

    # Iterate over all intended_steps groups
    for intended_step, group in grouped:
        # For each function, calculate its accuracy within the current intended_steps group
        accuracy_per_function = {}
        for function_name in function_map.keys():
            accuracy = calculate_function_accuracy(group, function_map, function_name)
            accuracy_per_function[function_name] = accuracy

        # Store the accuracies for each function within this LLM
        accuracies[intended_step] = accuracy_per_function

    accuracy_per_function = {}
    for function_name in function_map.keys():
        accuracy = calculate_function_accuracy(df, function_map, function_name)
        accuracy_per_function[function_name] = accuracy
    accuracies['Total'] = accuracy_per_function

    # Convert the dictionary to a DataFrame for easier handling
    accuracies_df = pd.DataFrame(accuracies)


    return accuracies_df

# Calculate the function accuracies per LLM
df_calculate_function_accuracy_per_inteded_step = calculate_function_accuracy_per_llm(df, function_map)
df_calculate_function_accuracy_per_inteded_step

,gpt-3.5-turbo,gpt-4-turbo,Total
function_fruit_and_veg_offers,0.700000,1.000000,0.850000
function_wine_selection_or_offers,1.000000,1.000000,1.000000
function_fridge_items,0.750000,0.750000,0.750000
function_date,0.789474,0.947368,0.868421
function_get_recipe_info,0.625000,0.933333,0.774194
function_find_recipe,0.653846,0.884615,0.769231
function_dish_pairing_for_wine,0.600000,1.000000,0.800000
function_wine_pairing,0.750000,1.000000,0.875000
function_image_classification,1.000000,1.000000,1.000000
function_compute_shopping_list,0.750000,1.000000,0.875000


In [67]:
#TODO: add to W&B

In [68]:
# create a copy of with rounded values
temp_df = df_calculate_function_accuracy_per_inteded_step.copy().round(2)
# transpose temp_df
temp_df = temp_df.transpose()
# change all parameter names to remove 'parameter_'
temp_df.columns = [col[9:] for col in temp_df.columns]
fig_accuracy_func_steps = px.imshow(temp_df,
                labels=dict(x="Function", y="Intended Steps", color="Accuracy"),
                x=temp_df.columns,
                y=temp_df.index,
                aspect="auto",
                color_continuous_scale='blugrn',
                text_auto=True)  

fig_accuracy_func_steps.update_layout(
    title="Parameter Accuracy per Intended Steps")
fig_accuracy_func_steps.update_xaxes(showgrid=False)
fig_accuracy_func_steps.update_yaxes(showgrid=False)
fig_accuracy_func_steps.show()

### Logging graphs

### Error Analysis

In [70]:

# if possible, split value in 'domain' by comma and keep the first value
# if not possible, keep the original value
# store the results in a new column called 'dtemp_error'
# df['temp_error'] = df['domain'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else x)

df.error.unique()

array([nan, 'task_misalignment', 'hallucination_date', 'early_stopping',
       'hallucination_parameter', 'translation', 'missing_crucial_step',
       'required_parameters_not_set', 'api_behaviour'], dtype=object)

In [71]:
def calculate_errors_by_step_and_type(df, function_map, llm_column='LLM', steps_column='intended_steps', error_column='error'):
    # Initialize a dictionary to hold the errors data
    errors_data = {}
    
    # If Overall is required, add it to the unique LLMs
    unique_llms = df[llm_column].unique().tolist() + ['Overall']

    for llm in unique_llms:
        # If not 'Overall', filter the DataFrame for the current LLM
        if llm != 'Overall':
            df_llm = df[df[llm_column] == llm]
        else:
            df_llm = df

        # Initialize dictionary for this LLM
        errors_data[llm] = {}

        # Get unique values for intended steps and error types
        unique_steps = df_llm[steps_column].unique()
        unique_errors = df_llm[error_column].unique()

        # Loop through intended steps and error types to count errors
        for step in unique_steps:
            df_step = df_llm[df_llm[steps_column] == step]
            errors_data[llm][step] = {error: (df_step[error_column] == error).sum() for error in unique_errors}

    # Convert the dictionary to a DataFrame
    errors_df = pd.DataFrame.from_dict(
        {(llm, step): errors_data[llm].get(step, {})
         for llm in unique_llms for step in unique_steps},
        orient='index'
    ).fillna(0)  # Fill missing values with 0

    # Name the multi-index and reset the index to turn it into columns
    errors_df.index.names = ['LLM', 'Intended Step']
    errors_df.reset_index(inplace=True)
    
    return errors_df

# Calculate the errors
errors_df = calculate_errors_by_step_and_type(df, function_map, error_column='error')
errors_df.tail(2)

,LLM,Intended Step,NaN,task_misalignment,hallucination_date,early_stopping,hallucination_parameter,translation,missing_crucial_step,api_behaviour,required_parameters_not_set
13,Overall,4,0,2,6,1.0,0.0,2,2.0,1.0,1.0
14,Overall,5,0,1,0,0.0,0.0,0,0.0,0.0,0.0


In [72]:
# change to integers for all columns except 'LLM'
errors_df = errors_df.astype({col: int for col in errors_df.columns if col != 'LLM'})

# TODO: check later if useful
# remove 'NaN' column
errors_df = errors_df.drop(columns=np.nan)
errors_df.head(2)

,LLM,Intended Step,task_misalignment,hallucination_date,early_stopping,hallucination_parameter,translation,missing_crucial_step,api_behaviour,required_parameters_not_set
0,gpt-3.5-turbo,1,1,0,0,0,0,0,0,0
1,gpt-3.5-turbo,2,0,1,0,0,0,0,0,0


In [73]:
# filter for gpt-3.5-turbo
df_gpt_3_5 = errors_df[errors_df['LLM'] == 'gpt-3.5-turbo']
df_gpt_3_5 = df_gpt_3_5.sort_values(by='Intended Step')
fig_errors_gpt_3_5 = px.bar(df_gpt_3_5, x="Intended Step", y = df_gpt_3_5.columns[2:].tolist(), title="Error Analysis GPT 3.5-turbo")
fig_errors_gpt_3_5.show()

In [74]:
# filter for gpt-4
df_gpt_4 = errors_df[errors_df['LLM'] == 'gpt-4-turbo']
df_gpt_4 = df_gpt_4.sort_values(by='Intended Step')
fig_errors_gpt_4 = px.bar(df_gpt_4, x="Intended Step", y = df_gpt_4.columns[2:].tolist(), title="Error Analysis GPT 4-Turbo")
fig_errors_gpt_4.show()

In [75]:
df.tail(2)

,Unnamed: 0,ID,CONTROL_PROMPT_ID,user_prompt,LLM,intended_steps,steps,summary_flag,completion,correct_functions,...,parameter_dish_name,used_dish_name,function_image_classification,used_image_classification,parameter_image_url,used_image_url,function_compute_shopping_list,used_compute_shopping_list,parameter_items_list,used_items_list
68,68,69,34,"Using veggies that are currently on offer, ple...",gpt-3.5-turbo,4,8,False,False,False,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,69,97,34,"Using veggies that are currently on offer, ple...",gpt-4-turbo,4,10,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
# create df_errors_llm, group by 'LLM' and 'error' and count the number of errors
df_errors_llm = df.groupby(['LLM', 'error']).size().reset_index(name='count')
fig_errors_llm = px.bar(df_errors_llm, x = 'error', y='count', color='LLM',
                    title='Error Analysis',
                    text_auto=True,
                    # change order of bars:
                    category_orders={"error": ["hallucination", "external_error", "no_info_extraction", "required_parameters_not_set", "translation", "early_stopping"]},
                    )
fig_errors_llm.update_xaxes(title_text='Error Type')
fig_errors_llm.update_yaxes(title_text='Occurences')
fig_errors_llm.show()


In [77]:
# plot the occurences of 'error' in df as a bar plot
fig_errors = px.bar(df['error'].value_counts(),
                    title='Error Analysis',
                    text_auto=True)
fig_errors.update(layout_showlegend=True)
fig_errors.update_xaxes(title_text='Error Type')
fig_errors.update_yaxes(title_text='Occurences')
fig_errors.show()